In [1]:
import numpy as np
import cv2
import time

In [2]:
print("Starting")

Starting


In [3]:
image_bgr = cv2.imread('PH.jpg')


In [4]:
h, w = image_bgr.shape[:2]  # Slicing from tuple only first two elements

In [5]:
blob = cv2.dnn.blobFromImage(image_bgr, 1 / 255.0, (416, 416),
                             swapRB=True, crop=False)


In [ ]:
with open('classes.name') as f:
    # creating list out of names
    labels = [line.strip() for line in f]
network = cv2.dnn.readNetFromDarknet('yolov3_custom.cfg',
                                     'C:/Users/hp/OneDrive/Desktop/Project/weights/yolov3_custom_final.weights')

layers_names_all = network.getLayerNames()

layers_names_output = \
    [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]


probability_minimum = 0.1
threshold = 0.3
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')


network.setInput(blob)  # setting blob as input to the network
start = time.time()
output_from_network = network.forward(layers_names_output)
end = time.time()

# Showing spent time for forward pass
print('Objects Detection took {:.5f} seconds'.format(end - start))

bounding_boxes = []
confidences = []
class_numbers = []

for result in output_from_network:
    for detected_objects in result:
        
        scores = detected_objects[5:]
        
        class_current = np.argmax(scores)
        
        confidence_current = scores[class_current]
        if confidence_current > probability_minimum:
            
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            
            x_center, y_center, box_width, box_height = box_current
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)



results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                           probability_minimum, threshold)
counter = 1

# Checking if there is at least one detected object after non-maximum suppression
if len(results) > 0:
    c=0
    # Going through indexes of results
    for i in results.flatten():
        # Showing labels of the detected objects
        print('Object {0}: {1}'.format(counter, labels[int(class_numbers[i])]))
        c=c+1;
        # Incrementing counter
        counter += 1
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

        colour_box_current = colours[class_numbers[i]].tolist()
        cv2.rectangle(image_bgr, (x_min, y_min),
                      (x_min + box_width, y_min + box_height),
                      colour_box_current, 2)

        # Preparing text with label and confidence for current bounding box
        text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                               confidences[i])

        # Putting text with label and confidence on the original image
        cv2.putText(image_bgr, text_box_current, (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)



print()
print("total",c)
#print('Total objects been detected:', len(bounding_boxes))
#print('Number of objects left after non-maximum suppression:', counter - 1)

cv2.namedWindow('Detections', cv2.WINDOW_NORMAL)
# Pay attention! 'cv2.imshow' takes images in BGR format
cv2.imshow('Detections', image_bgr)
# Waiting for any key being pressed
cv2.waitKey(0)
# Destroying opened window with name 'Detections'
cv2.destroyWindow('Detections')

Objects Detection took 2.83240 seconds
Object 1: slight
Object 2: slight
Object 3: slight
Object 4: slight

total 4
